
Facial Recognition using PyTorch and OpenCV

https://ritik12.medium.com/facial-recognition-using-pytorch-and-opencv-467c4e41d1f


Machine Learning - Face Recognition CNN Pytorch.ipynb
https://github.com/rubencg195/Pytorch-Tutorials/blob/master/Machine%20Learning%20-%20Face%20Recognition%20CNN%20Pytorch.ipynb



Face Recognition Using Pytorch
https://github.com/timesler/facenet-pytorch

Face Landmarks Detection With PyTorch

https://towardsdatascience.com/face-landmarks-detection-with-pytorch-4b4852f5e9c4



다중입력 deep neural network
https://rosenfelder.ai/multi-input-neural-network-pytorch/



Understanding dimensions in PyTorch
https://towardsdatascience.com/understanding-dimensions-in-pytorch-6edf9972d3be

학습하기 
https://github.com/deeplearningzerotoall/PyTorch

In [1]:
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor

from FaceFeatureDataset import FaceFeatureDataset

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Using cuda device


In [2]:
batch_size = 30
epochs = 1000

In [3]:
#linear_layer_2d = nn.Linear(in_features=68, out_features=128)
#flatten = nn.Flatten()
#input_2d = torch.randn(2, 2, 68) # n, channel, features
#print(input_2d)
#print(input_2d.shape)
#output_2d = linear_layer_2d(input_2d)
#print('output_2d' , output_2d)
#print(output_2d.size())

#output_1d = flatten(input_2d)
#print(output_1d.size())
#linear_layer_2d(output_1d)
#print("1D ",  output_1d)

#test_sequence = nn.Sequential(nn.Linear(64, 32), nn.ReLU(), nn.Linear(32,10))
#out = test_sequence(input_2d)
#print(out)

In [4]:
'''
testload = pd.read_csv("./outimg/Train/facefeature.csv")
landmarks = np.array(testload.iloc[0, 1:])
landmarks = landmarks.astype('float').reshape(-1, 2)
print(landmarks)

dataiter = iter(testload)
landmark = next(dataiter)
print(landmark)
# print(landmarks.shape)
'''

'\ntestload = pd.read_csv("./outimg/Train/facefeature.csv")\nlandmarks = np.array(testload.iloc[0, 1:])\nlandmarks = landmarks.astype(\'float\').reshape(-1, 2)\nprint(landmarks)\n\ndataiter = iter(testload)\nlandmark = next(dataiter)\nprint(landmark)\n# print(landmarks.shape)\n'

In [5]:
# Create DataLoader 

training_data = FaceFeatureDataset(feature_file="./outimg/Train/facefeature.csv", label_file="./Dataset/Train/csv/train.csv")
test_data = FaceFeatureDataset(feature_file="./outimg/Test/facefeature.csv", label_file="./Dataset/Test/csv/test.csv")

train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# 데이터 로드 확인 
for X, y in train_loader:
    print(f"Shape of X [N, F, C]: {X.shape}") # N , Channel, H= width W = height
    print(f"Shape of Tensor y: {y.shape} {y.dtype}")       
    break

n_total_steps = len(train_loader)
# print(f'Traing dat length {n_total_steps}')
# print(y)
print('X type', X.dtype)
print('y type', y.dtype)


Shape of X [N, F, C]: torch.Size([30, 136])
Shape of Tensor y: torch.Size([30, 33]) torch.float32
X type torch.float32
y type torch.float32


In [6]:
#
#dataiter = iter(train_loader)
#landmark, labels = next(dataiter)
#print("landmark Shape ", landmark.shape)
#print(landmark)
#flatten = nn.Flatten()
#linear1 = nn.Linear(68, 32)
#x = flatten(landmark)
#print(x)
#print(x.size())
# print(landmark.shape)
# print('linear1', x.shape)


In [7]:

num_classes = 33

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(68 * 2, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
            #nn.ReLU(),
        )
    

    def forward(self, x):
        # x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=136, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=33, bias=True)
  )
)


In [8]:
#criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)        
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print('batch',  batch)
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
# Training
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, criterion, optimizer)    
print("Done!")

Epoch 1
-------------------------------
loss: 0.280013  [    0/  100]
Epoch 2
-------------------------------
loss: 0.274052  [    0/  100]
Epoch 3
-------------------------------
loss: 0.273271  [    0/  100]
Epoch 4
-------------------------------
loss: 0.285158  [    0/  100]
Epoch 5
-------------------------------
loss: 0.273536  [    0/  100]
Epoch 6
-------------------------------
loss: 0.267302  [    0/  100]
Epoch 7
-------------------------------
loss: 0.271044  [    0/  100]
Epoch 8
-------------------------------
loss: 0.268517  [    0/  100]
Epoch 9
-------------------------------
loss: 0.266093  [    0/  100]
Epoch 10
-------------------------------
loss: 0.260669  [    0/  100]
Epoch 11
-------------------------------
loss: 0.256062  [    0/  100]
Epoch 12
-------------------------------
loss: 0.257562  [    0/  100]
Epoch 13
-------------------------------
loss: 0.260997  [    0/  100]
Epoch 14
-------------------------------
loss: 0.251173  [    0/  100]
Epoch 15
------

In [11]:
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    print('test size', size )
    # num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)            
            print('pred =', pred)
            #print('loss', loss)
            #print('real', y)



In [13]:

test(test_loader, model, criterion)

print("Done !")

test size 100
pred = tensor([[0.4919, 0.4863, 0.4918, 0.5115, 0.5185, 0.5222, 0.4741, 0.5019, 0.5185,
         0.5130, 0.5202, 0.4985, 0.4952, 0.5160, 0.4838, 0.4896, 0.5145, 0.5347,
         0.4939, 0.5224, 0.4991, 0.5050, 0.4965, 0.4989, 0.5211, 0.4873, 0.5250,
         0.4899, 0.4856, 0.5042, 0.5261, 0.5184, 0.4844],
        [0.4921, 0.4861, 0.4942, 0.5106, 0.5202, 0.5203, 0.4762, 0.5033, 0.5195,
         0.5124, 0.5219, 0.4995, 0.4943, 0.5147, 0.4818, 0.4933, 0.5126, 0.5351,
         0.4959, 0.5223, 0.4987, 0.5068, 0.4977, 0.5004, 0.5207, 0.4904, 0.5271,
         0.4891, 0.4843, 0.5055, 0.5256, 0.5188, 0.4868],
        [0.4939, 0.4890, 0.4969, 0.5138, 0.5223, 0.5254, 0.4767, 0.5050, 0.5227,
         0.5157, 0.5240, 0.5013, 0.4978, 0.5191, 0.4871, 0.4931, 0.5169, 0.5379,
         0.4977, 0.5245, 0.5009, 0.5093, 0.4995, 0.5015, 0.5251, 0.4904, 0.5279,
         0.4939, 0.4885, 0.5090, 0.5306, 0.5236, 0.4879],
        [0.4948, 0.4868, 0.4939, 0.5129, 0.5195, 0.5230, 0.4771, 0.5033, 0.5

In [15]:

model.eval()
x = torch.Tensor([-0.36369685665697504,-0.21546217565417758,-0.3574112677601623,-0.12973942618903955,-0.34360908442591476,-0.041386801250754224,-0.32728893521312186,0.047003126885731494,-0.3008596192879478,0.13302430193647485,-0.24910143178451916,0.20934664545504422,-0.1820489330188162,0.27330297877009146,-0.1047753579461298,0.32733666136376105,-0.016907674584453414,0.3462680344506007,0.0714076471556312,0.3349838169948983,0.15768994459377905,0.290928739919908,0.23164353502660584,0.22906138570409784,0.28812057710241923,0.15686104558670244,0.32215974546052994,0.06921718141423006,0.3461270503044599,-0.018575895551044996,0.3676136924680454,-0.1089242415930659,0.3790284711174502,-0.19942180042788948,-0.29962861374732563,-0.29006857205551595,-0.25378298315870307,-0.3246486367875362,-0.19323989247901685,-0.3313072576663558,-0.13799385594382574,-0.3204147237917603,-0.08297163859783863,-0.2944143946458938,0.0857693784628889,-0.2944330462449941,0.146536288331779,-0.3161994623950847,0.20970925448461253,-0.330374677711339,0.27513906412858635,-0.3268868286795764,0.32741949640682433,-0.2858160074606396,0.00400076800702176,-0.21505184047396997,0.0031054912502057608,-0.1546206593888858,-0.0002704481869548747,-0.09674474738054728,-0.0037209940205167324,-0.033832903615118703,-0.07230292391244719,0.005447364090185869,-0.03971858028416264,0.01600416918097547,-0.004616270777332843,0.02659827746996546,0.0307844643151024,0.017048658730594135,0.06366723352899217,0.007461736793021978,-0.2304684842832848,-0.19837731087827104,-0.19228866092489982,-0.22551538756925782,-0.14192934335399632,-0.22476932360524426,-0.10212683087388219,-0.19143891601294627,-0.14767403587689942,-0.17700257830928734,-0.19803335344780293,-0.17774864227330067,0.10182840528827686,-0.1884173569586921,0.14263810411980915,-0.22307202808711402,0.19558999396565901,-0.22732459268199012,0.23802238191892033,-0.20151077952712704,0.1999171649569368,-0.17940863459323053,0.1470025783092873,-0.17767403587689945,-0.137005321191508,0.12285918042679256,-0.08882824071534368,0.10090624828569872,-0.04324373251412583,0.08395194470349432,-0.008104119809095423,0.09202808711393928,0.027259312085139098,0.0849964342531132,0.07483954139009275,0.10333095616874222,0.1299363651324813,0.12429535355751808,0.07649953371002238,0.16128147457348174,0.030989631905205983,0.17319984639859565,-0.006854462669372952,0.17767623018267586,-0.04706731032969447,0.17204344725437493,-0.08961160787755773,0.15378353173514736,-0.11182566240605629,0.12323221240879911,-0.04624663996927969,0.11664819792638104,-0.008514454989302811,0.11972571177793634,0.026774370508530265,0.11772999067420054,0.10723736902737391,0.1264775906522574,0.0292550331888749,0.1202852597509464,-0.006071095507158897,0.12479894673322722,-0.043765977288935165,0.1192034670031269]).to(device)
with torch.no_grad():
    pred = model(x)
    print(f'{pred} {pred.shape}')    


tensor([0.4830, 0.4763, 0.4866, 0.5018, 0.5129, 0.5121, 0.4688, 0.4941, 0.5137,
        0.5043, 0.5141, 0.4931, 0.4867, 0.5052, 0.4748, 0.4861, 0.5047, 0.5258,
        0.4892, 0.5125, 0.4907, 0.4991, 0.4896, 0.4925, 0.5105, 0.4842, 0.5184,
        0.4801, 0.4732, 0.4998, 0.5166, 0.5106, 0.4792], device='cuda:0') torch.Size([33])
